In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',1000)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

*RKL
*John Hopkins
*Rest APIs

## Goals

* We would like to understand the data quality

* Everything should be automated as much as possible:
    how many clicks do we need to execute the full pipeline
    

## Contraints:

* Each notebook should be left clean and ready for full execution

In [ ]:
datapath = '../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw =pd.read_csv(datapath)

In [ ]:
pd_raw

In [ ]:
pd_raw.columns[4:]

In [ ]:
time_idx = pd_raw.columns[4:]

In [ ]:
df_plot = pd.DataFrame({'date':time_idx})
df_plot.head()

In [ ]:
pd_raw['Country/Region']

In [ ]:
len(pd_raw['Country/Region'].unique())

In [ ]:
pd_raw[pd_raw['Country/Region']=='United Kingdom'].iloc[:,4::].sum(axis=0)

In [ ]:
final_data_list=[]

In [ ]:
country_list =('Spain','Germany','Australia','Italy','China'     
                             )

In [ ]:
for each in country_list:
    df_plot[each]= np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))

In [ ]:
%matplotlib inline
df_plot.head()

In [ ]:
%matplotlib inline
df_plot.set_index('date').plot()

In [ ]:
type(df_plot.date[0])

In [ ]:
df_plot.head()


In [ ]:
from datetime import datetime

In [ ]:
df_plot.date[0]
    


In [ ]:
datetime.strptime(df_plot.date[0],"%m/%d/%y")

In [ ]:
time_index =[datetime.strptime(each, "%m/%d/%y") for each in df_plot.date]
time_index[0:5]

In [ ]:
time_str = [datetime.strftime(each, "%y-%m-%d") for each in time_index]
time_str[0:5]

In [ ]:
df_plot['date']=time_index
type(df_plot['date'][0])

In [ ]:
df_plot.head()

In [ ]:
df_plot.to_csv('../data/processed/COVID_small_flat_table.csv', sep=';')

In [ ]:
##Relational data model -defining PK

In [ ]:
pd_raw =pd.read_csv(datapath)
pd_raw.head()

In [ ]:
pd_data_base=pd_raw.rename(columns={'Province/State':'state', 'Country/Region': 'country'})

In [ ]:
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

In [ ]:
pd_data_base.set_index(['state','country'])

In [ ]:
test_pd =pd_data_base.set_index(['state','country']).T

In [ ]:
test_pd.columns

In [ ]:
test_pd.stack(level=[0,1]).reset_index().rename(columns={'level_o':'date','0':'confirmed'})

In [ ]:
test_pd.dtypes

In [ ]:
test_pd.to_csv('../data/processed/COVID_relational_data.csv', sep=';')

## Group-by apply

In [58]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,Kosovo,0.0
2,2020-01-22,no,Kuwait,0.0
3,2020-01-22,no,Kyrgyzstan,0.0
4,2020-01-22,no,Laos,0.0


In [59]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2022-01-01')]


In [60]:
len(test_data)

270

In [61]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2022-05-16,no,25818405.0
US,2022-05-16,no,82613620.0


In [62]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

the test slope is: [2.]


In [63]:
test_data1=test_data.groupby(['state','country']).agg(np.max)
test_data1

date   confirmed
state country                       
no    Germany 2022-05-16  25818405.0
      US      2022-05-16  82613620.0

In [ ]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression1(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''
    #print(in_array)
    #print(type(in_array))
    y = np.array(in_array['confirmed'])
    X = np.arange(len(y)).reshape(-1, 1)
    #y = np.array(in_array)
    #X = np.arange(-1,2).reshape(-1, 1)

    #assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

#if __name__ == '__main__':
  #  test_data_reg=np.array([2,4,6])
   # result=get_doubling_time_via_regression(test_data_reg)
   # print('the test slope is: '+str(result))

In [ ]:
# this command will only work when adapting the get_doubling_time_via_regression function

test_data.groupby(['state','country']).apply(get_doubling_time_via_regression1)

In [64]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [65]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country        
no     Germany  202675            NaN
                202959            NaN
                203244     293.887868
                203529     162.012190
                203814     118.442367
                204100     121.963815
                204384     132.197147
                204669     161.561176
                204954     242.906801
                205239     212.629577
                205524     120.408516
                205809      94.683745
                206095      89.228634
                206380      91.993507
                206664     121.124787
                206949     183.726467
                207234     147.647020
                207520      86.619750
                207805      66.655449
                208090      60.816336
                208375      61.381242
                208659      77.675822
                208944     116.561488
                209229      92.105274
                209514      61.068848
                209799     

In [66]:
test_data

,date,state,country,confirmed
202675,2022-01-02,no,Germany,7189329.0
202738,2022-01-02,no,US,55310349.0
202959,2022-01-03,no,Germany,7207847.0
203009,2022-01-03,no,US,56355269.0
203244,2022-01-04,no,Germany,7238408.0
203308,2022-01-04,no,US,57167433.0
203529,2022-01-05,no,Germany,7297320.0
203579,2022-01-05,no,US,57830005.0
203814,2022-01-06,no,Germany,7361660.0
203864,2022-01-06,no,US,58665038.0


In [67]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [68]:
pd_DR_result.head()

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [69]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1069,NaN
4,Alberta,Canada,1333,NaN


In [71]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,level_0,index,date,state,country,confirmed
0,0,0,2020-01-22,Alberta,Canada,0.0
1,1,1,2020-01-22,no,Kosovo,0.0
2,2,2,2020-01-22,no,Kuwait,0.0
3,3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,4,2020-01-22,no,Laos,0.0


In [72]:
#del pd_JH_data['level_0']

In [73]:
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,Kosovo,0.0
2,2,2020-01-22,no,Kuwait,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,2020-01-22,no,Laos,0.0


In [74]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,Kosovo,0.0,NaN
2,2,2020-01-22,no,Kuwait,0.0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN
4,4,2020-01-22,no,Laos,0.0,NaN


In [ ]:
#pd_result_larg[pd_result_larg['country']=='Germany']

# Filtering the data with groupby apply 

In [75]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [76]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [77]:
pd_filtered_result.head()

,index,state,country,confirmed,confirmed_filtered
0,0,Alberta,Canada,0.0,0.0
1,1,no,Kosovo,0.0,0.0
2,2,no,Kuwait,0.0,0.0
3,3,no,Kyrgyzstan,0.0,0.0
4,4,no,Laos,0.0,0.0


In [78]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0
4,4,2020-01-22,no,Laos,0.0,NaN,0.0


# Filtered doubling rate

In [79]:
pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
241105,no,Zimbabwe,239826,1716.849781
241106,no,Zimbabwe,240039,2017.064290
241107,no,Zimbabwe,240346,1840.654841
241108,no,Zimbabwe,240588,1895.517941
241109,no,Zimbabwe,241109,2371.917222


In [80]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
241105,241105,2022-05-16,no,Andorra,42156.0,inf,42156.0,-5.793877e+15
241106,241106,2022-05-16,no,Algeria,265823.0,7.594829e+04,265822.6,7.818212e+04
241107,241107,2022-05-16,no,Albania,275621.0,1.172780e+04,275636.4,7.807397e+03
241108,241108,2022-05-16,no,Argentina,9135308.0,5.368783e+02,9135308.0,8.949083e+02
241109,241109,2022-05-16,no,Zimbabwe,249431.0,2.215831e+03,249393.6,2.371917e+03


In [81]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [82]:
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN
1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN
2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN
4,4,2020-01-22,no,Laos,0.0,NaN,0.0,NaN


In [83]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
239724,239724,2022-05-12,no,Germany,25661838.0,323.957774,25642420.0,342.144097
240009,240009,2022-05-13,no,Germany,25723697.0,392.172553,25688075.0,465.049865
240295,240295,2022-05-14,no,Germany,25729848.0,755.922002,25733188.2,566.010880
240580,240580,2022-05-15,no,Germany,25732153.0,6085.280511,25765347.2,665.928242
240864,240864,2022-05-16,no,Germany,25818405.0,581.775248,25797506.2,801.186206


In [84]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)